# adding requirements

In [2]:
!pip3 install gensim
!pip3 install pandas
!pip3 install nltk

     |████████████████████████████████| 24.2 MB 84 kB/s  eta 0:00:01
     |████████████████████████████████| 113 kB 893 kB/s eta 0:00:01
     |████████████████████████████████| 25.8 MB 2.4 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107092 sha256=4d478dc52e0c1c0a8be3538f3d46eb12dc1832925504a6620ced768b98fdf4e3
  Stored in directory: /root/.cache/pip/wheels/11/73/9a/f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open
     |████████████████████████████████| 9.3 MB 104 kB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 667 kB/s eta 0:00:01
     |████████████████████████████████| 301 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 679 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 647 kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=068a2e2395ffdbd5280ef69c22876d0b9304b26b561b2f0daa47ccf3dd22d176
  Stored in d

In [7]:
import pickle
with open("mention_overall_dict.pickle", "rb") as f:
    mention_to_entities = pickle.load(f)
print ("all ready!")


all ready!


# example of codes which will be use in making data set

In [6]:
from nltk.util import everygrams
s_words=[ ' '.join(grams) for grams in list(everygrams('A class is a blueprint for the object.'.split()))]
print( s_words)


['A', 'class', 'is', 'a', 'blueprint', 'for', 'the', 'object.', 'A class', 'class is', 'is a', 'a blueprint', 'blueprint for', 'for the', 'the object.', 'A class is', 'class is a', 'is a blueprint', 'a blueprint for', 'blueprint for the', 'for the object.', 'A class is a', 'class is a blueprint', 'is a blueprint for', 'a blueprint for the', 'blueprint for the object.', 'A class is a blueprint', 'class is a blueprint for', 'is a blueprint for the', 'a blueprint for the object.', 'A class is a blueprint for', 'class is a blueprint for the', 'is a blueprint for the object.', 'A class is a blueprint for the', 'class is a blueprint for the object.', 'A class is a blueprint for the object.']


In [10]:
mention_to_entities["Toni Morrison"]

Counter({'Toni%20Morrison': 91})

In [12]:
mention_to_entities["Morrison"]

Counter({'John%20Robert%20Morrison': 1,
         'Herbert%20Morrison': 1,
         'Morrison%2C%20Virginia': 1,
         'Morrison%2C%20Illinois': 1,
         'Morrison%2C%20Colorado': 9,
         'Morrison%20County%2C%20Minnesota': 3,
         'Morrison%20%28community%29%2C%20Wisconsin': 1,
         'Morrison%2C%20Missouri': 1,
         '%23Morrison%20shelter': 1,
         'Bruce%20Morrison': 1,
         'Morrison%20Bridge': 2,
         'Morrison%20Formation': 1,
         'Morrison%2C%20OK': 1,
         'Morrison%20%28surname%29': 1,
         'Morrison%20Government': 1})

In [11]:
mention_to_entities["Toni"]

Counter({'Toni%20%281935%20film%29': 2,
         'Antonio%20Mu%C3%B1oz%20G%C3%B3mez': 2,
         'Toni%20Adams': 1,
         'Toni%20%28footballer%2C%20born%201946%29': 2,
         'Toni%20Tennille': 1,
         'Toni%20Braxton': 1})

# dataset maker

In [14]:
from gensim import utils
import json
import pandas as pd
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import time
from urllib.parse import quote
from nltk.util import everygrams

In [17]:


file_object = open('results/segment0.txt','a+', encoding='utf-8')

start_time = time.time()
lineCount=0

 # iterate over the plain text data we just created
with utils.open('enwiki-latest.json.gz', 'rb') as f:
     for line in range(70000):
#      for line in f:
            lineCount+=1;
            # segmenting results to batches with size of 100000
            # to avoid lost of  all articles for an unexcpected error
            if(lineCount%100000==0):
                file_object.close()
                name="segment"+str(lineCount/100000)+""
                file_object = open('results/'+name+'.txt','a+', encoding='utf-8')
                
            # decode each JSON line into a Python dictionary object (each line is one article)
            article = json.loads(next(f))
            # each article has:
            # title
            # inter links
            # sections 
            title=article['title']
            p_links= article['interlinks']
            all_links_till_here=set()
            # link[1] refers to metion of entitiy in text
            # link[0] refers to the entity
            for link in p_links:
                link.append([ ' '.join(grams) for grams in list(everygrams(link[1].split()))])
            
            for section_text in article['section_texts']:
                sentences = sent_tokenize(section_text)
                for sentence in sentences:
                    entities=set()
                    #first try to find entities which are mentioned  by wiki in sentence
                    for link in p_links:
                        if(link[1] in sentence):
                            all_links_till_here.add(link[0])
                            entities.add(link[0].replace(' ','_'))
                        #try to find entities which arent mentioned  by wiki for avoidance of reputation
                        # mention_to_entities contains collection of all entities that are exists for a mention
                        for ngram_link in link[2]:
                            if(" "+ngram_link+" " in sentence):
                                try:
                                    if((quote(link[0])in mention_to_entities[ngram_link]) and (link[0] in all_links_till_here)):
                                        entities.add(link[0].replace(' ','_'))
                                        all_links_till_here.add(link[0])
                                except:
                                    pass

                    if(len(entities)>0):
                        # writing each triple in a line (from_entity,sentence,to_entities)
                        file_object.write("from_entity:"+title+"")
                        file_object.write('\n')
                        file_object.write("sentence:"+sentence)
                        file_object.write('\n') 
                        file_object.write("to_entities:"+",".join(entities))
                        file_object.write('\n')
                        
             
 

elapsed_time = time.time() - start_time
since_beginning = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Since beginning: %s" % (since_beginning))





Since beginning: 00:46:32


# example


In [1]:
import pickle
with open("mention_overall_dict.pickle", "rb") as f:
    mention_to_entities = pickle.load(f)
print ("all ready!")

all ready!


In [2]:
from gensim import utils
import json
import pandas as pd
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import time
from urllib.parse import quote
from nltk.util import everygrams

In [5]:


file_object = open('dataset-making-example/results-example/Beloved.txt','a+', encoding='utf-8')

start_time = time.time()
lineCount=0

 # iterate over the plain text data we just created
with utils.open('dataset-making-example/Beloved.json.gz', 'rb') as f:
    
     for line in range(1):   
            # decode each JSON line into a Python dictionary object (each line is one article)
            article = json.loads(next(f))
            # each article has:
            # title
            # inter links
            # sections 
            title=article['title']
            p_links= article['interlinks']
            all_links_till_here=set()
            # link[1] refers to metion of entitiy in text
            # link[0] refers to the entity
            for link in p_links:
                link.append([ ' '.join(grams) for grams in list(everygrams(link[1].split()))])
            for section_text in article['section_texts']:
                sentences = sent_tokenize(section_text)
                for sentence in sentences:
                    entities=set()
                    #first try to find entities which are mentioned  by wiki in sentence
                    for link in p_links:
                        if(link[1] in sentence):
                            all_links_till_here.add(link[0])
                            entities.add(link[0].replace(' ','_'))
                        #try to find entities which arent mentioned  by wiki for avoidance of reputation
                        # mention_to_entities contains collection of all entities that are exists for a mention
                        for ngram_link in link[2]:
                            if(" "+ngram_link+" " in sentence):
                                print("\n")
                                print("----------------ngram_link:--------------")
                                print(ngram_link)
                                print("----------------sentence:----------------")
                                print(sentence)
                                print("----------------checking if  ngram_link in sentence:---------")
                                print(ngram_link in sentence)
                                try:
                                    print("----------------mention to entity result:-----------------")
                                    print(mention_to_entities[ngram_link])
                                    if((quote(link[0])in mention_to_entities[ngram_link]) and (link[0] in all_links_till_here)):
                                        entities.add(link[0].replace(' ','_'))
                                        all_links_till_here.add(link[0])
                                except:
                                    pass

                    if(len(entities)>0):
                        # writing each triple in a line (from_entity,sentence,to_entities)
                        file_object.write("from_entity:"+title+"")
                        print("from_entity:"+title+"")
                        file_object.write('\n')
                        print('\n')
                        file_object.write("sentence:"+sentence)
                        print(" sentence:"+sentence)
                        file_object.write('\n') 
                        print('\n')
                        file_object.write("to_entities:"+",".join(entities))
                        print("to_entities:"+",".join(entities))
                        file_object.write('\n')
                        print('\n')
                        file_object.close()
 

elapsed_time = time.time() - start_time
since_beginning = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Since beginning: %s" % (since_beginning))







----------------ngram_link:--------------
Toni
----------------sentence:----------------

	    
		
		
		
		'''''Beloved''''' is a 1987 novel by the American writer Toni Morrison.
----------------checking if  ngram_link in sentence:---------
True
----------------mention to entity result:-----------------
Counter({'Toni%20%281935%20film%29': 2, 'Antonio%20Mu%C3%B1oz%20G%C3%B3mez': 2, 'Toni%20%28footballer%2C%20born%201946%29': 2, 'Toni%20Adams': 1, 'Toni%20Tennille': 1, 'Toni%20Braxton': 1})


----------------ngram_link:--------------
American
----------------sentence:----------------

	    
		
		
		
		'''''Beloved''''' is a 1987 novel by the American writer Toni Morrison.
----------------checking if  ngram_link in sentence:---------
True
----------------mention to entity result:-----------------
Counter({'United%20States': 7348, 'Americans': 2075, 'United%20States%20of%20America': 297, 'united%20states': 174, 'American%20people': 124, 'American%20Airlines': 84, 'American%20ethnicity':

ValueError: I/O operation on closed file.